In [1]:
import numpy as np
import pandas as pd

In [2]:
DNAfull = pd.read_csv('DNAfull.csv', sep = " ", index_col=0)
DNAfull

,A,T,G,C,S,W,R,Y,K,M,B,V,H,D,N
A,5,-4,-4,-4,-4,1,1,-4,-4,1,-4,-1,-1,-1,-2
T,-4,5,-4,-4,-4,1,-4,1,1,-4,-1,-4,-1,-1,-2
G,-4,-4,5,-4,1,-4,1,-4,1,-4,-1,-1,-4,-1,-2
C,-4,-4,-4,5,1,-4,-4,1,-4,1,-1,-1,-1,-4,-2
S,-4,-4,1,1,-1,-4,-2,-2,-2,-2,-1,-1,-3,-3,-1
W,1,1,-4,-4,-4,-1,-2,-2,-2,-2,-3,-3,-1,-1,-1
R,1,-4,1,-4,-2,-2,-1,-4,-2,-2,-3,-1,-3,-1,-1
Y,-4,1,-4,1,-2,-2,-4,-1,-2,-2,-1,-3,-1,-3,-1
K,-4,1,1,-4,-2,-2,-2,-2,-1,-4,-1,-3,-3,-1,-1
M,1,-4,-4,1,-2,-2,-2,-2,-4,-1,-3,-1,-1,-3,-1


In [3]:
right = "→"
down = "↓"
diagonal = "↘"

In [4]:
def S(grid, i, j, d, score):
    dirs = ["→", "↘", "↓"]
    s = [ grid[i, j - 1][1] - d, 
          grid[i - 1, j - 1][1] + score,
          grid[i - 1, j][1] - d ]
    return (dirs[s.index(max(s))], max(s))

def highlight_path(x):
    colored_df= x.copy()
    colored_df.loc[:,:] = '' 
    # back
    i = x.shape[0] - 1
    j = x.shape[1] - 1
    colored_df.iloc[:, :] = "background-color: ivory"
    colored_df.iloc[0, :] = "background-color: beige"
    colored_df.iloc[:, 0] = "background-color: beige"
    colored_df.iloc[0, :] = "background-color: beige"
    colored_df.iloc[1, 1] = "background-color: lightblue"
    while ( i != 1 or j != 1):
        colored_df.iloc[i, j] = "background-color: lightblue"
        if x.iloc[i, j][0] == right:
            j -= 1
        elif x.iloc[i, j][0] == down:
            i -= 1
        elif x.iloc[i, j][0] == diagonal:
            i -= 1
            j -= 1
    return colored_df

def needleman_wunsch(x, y, scor_mat, d):
    nucl_x = [""] + [ch for ch in x]
    nucl_y = [""] + [ch for ch in y]
    
    grid = np.empty(shape = (len(nucl_x), len(nucl_y)), dtype = object)
    for i, val in enumerate(range(0, -len(nucl_x) * d, -d)):
        grid[i, 0] = (down, val)
    for i, val in enumerate(range(0, -len(nucl_y) * d, -d)):
        grid[0, i] = (right, val)
    
    # forward 
    for i in range(1, len(nucl_x)):
        for j in range(1, len(nucl_y)):
            grid[i, j] = S(grid, i, j, d, scor_mat[nucl_x[i]][nucl_y[j]])
    # backward
    aligned_x = []
    aligned_y = [] 
    i = grid.shape[0] - 1
    j = grid.shape[1] - 1
    while ( i !=0 or j != 0):
        if grid[i, j][0] == right:
            aligned_x.append("-")
            aligned_y.append(nucl_y[j])
            j -= 1
        elif grid[i, j][0] == down:
            aligned_x.append(nucl_x[i])
            aligned_y.append("-")
            i -= 1
        elif grid[i, j][0] == diagonal:
            aligned_x.append(nucl_x[i])
            aligned_y.append(nucl_y[j])
            i -= 1
            j -= 1
    
    grid_df = pd.DataFrame(data=grid, index=range(len(nucl_x)), columns=range(len(nucl_y)))
    # insert X and Y as col and row
    row = {i : ch for i, ch in enumerate(nucl_y)}
    d = pd.DataFrame(row, index=["Y"])
    grid_df = pd.concat([d, grid_df], ignore_index=False)
    
    grid_df.insert(0, "X", [""] + nucl_x)
        
    grid_df = grid_df.style.apply(highlight_path, axis=None)
    return (aligned_x[::-1], aligned_y[::-1], grid_df)


In [5]:
x = "CTGTCTCCTG"
y = "ATGAGTCTCT"
aligned_x, aligned_y, grid = needleman_wunsch(x, y, DNAfull, 10)

In [6]:
grid

,X,0,1,2,3,4,5,6,7,8,9,10
Y,,,A,T,G,A,G,T,C,T,C,T
0,,"('→', 0)","('→', -10)","('→', -20)","('→', -30)","('→', -40)","('→', -50)","('→', -60)","('→', -70)","('→', -80)","('→', -90)","('→', -100)"
1,C,"('↓', -10)","('↘', -4)","('→', -14)","('→', -24)","('→', -34)","('→', -44)","('→', -54)","('↘', -55)","('→', -65)","('→', -75)","('→', -85)"
2,T,"('↓', -20)","('↘', -14)","('↘', 1)","('→', -9)","('→', -19)","('→', -29)","('→', -39)","('→', -49)","('↘', -50)","('→', -60)","('→', -70)"
3,G,"('↓', -30)","('↘', -24)","('↓', -9)","('↘', 6)","('→', -4)","('→', -14)","('→', -24)","('→', -34)","('→', -44)","('→', -54)","('→', -64)"
4,T,"('↓', -40)","('↘', -34)","('↘', -19)","('↓', -4)","('↘', 2)","('→', -8)","('↘', -9)","('→', -19)","('→', -29)","('→', -39)","('→', -49)"
5,C,"('↓', -50)","('↘', -44)","('↓', -29)","('↓', -14)","('↘', -8)","('↘', -2)","('→', -12)","('↘', -4)","('→', -14)","('→', -24)","('→', -34)"
6,T,"('↓', -60)","('↘', -54)","('↘', -39)","('↓', -24)","('↘', -18)","('↘', -12)","('↘', 3)","('→', -7)","('↘', 1)","('→', -9)","('→', -19)"
7,C,"('↓', -70)","('↘', -64)","('↓', -49)","('↓', -34)","('↘', -28)","('↘', -22)","('↓', -7)","('↘', 8)","('→', -2)","('↘', 6)","('→', -4)"
8,C,"('↓', -80)","('↘', -74)","('↓', -59)","('↓', -44)","('↘', -38)","('↘', -32)","('↓', -17)","('↘', -2)","('↘', 4)","('↘', 3)","('↘', 2)"


In [7]:
print("".join(aligned_y))
print("".join(aligned_x))

ATGAGTCTCT-
CTGTCTC-CTG


In [8]:
mat = pd.read_csv('mat_ones.csv', sep = " ", index_col=0)
x = "AGACTAGTTAC"
y = "CGAGACGT"
aligned_x, aligned_y, grid = needleman_wunsch(x, y, mat, 1)

In [9]:
grid

,X,0,1,2,3,4,5,6,7,8
Y,,,C,G,A,G,A,C,G,T
0,,"('→', 0)","('→', -1)","('→', -2)","('→', -3)","('→', -4)","('→', -5)","('→', -6)","('→', -7)","('→', -8)"
1,A,"('↓', -1)","('↘', -1)","('→', -2)","('↘', -1)","('→', -2)","('→', -3)","('→', -4)","('→', -5)","('→', -6)"
2,G,"('↓', -2)","('↘', -2)","('↘', 0)","('→', -1)","('↘', 0)","('→', -1)","('→', -2)","('→', -3)","('→', -4)"
3,A,"('↓', -3)","('↘', -3)","('↓', -1)","('↘', 1)","('→', 0)","('↘', 1)","('→', 0)","('→', -1)","('→', -2)"
4,C,"('↓', -4)","('↘', -2)","('↓', -2)","('↓', 0)","('↘', 0)","('↓', 0)","('↘', 2)","('→', 1)","('→', 0)"
5,T,"('↓', -5)","('↓', -3)","('↘', -3)","('↓', -1)","('↘', -1)","('↘', -1)","('↓', 1)","('↘', 1)","('↘', 2)"
6,A,"('↓', -6)","('↓', -4)","('↘', -4)","('↘', -2)","('↘', -2)","('↘', 0)","('↓', 0)","('↘', 0)","('↓', 1)"
7,G,"('↓', -7)","('↓', -5)","('↘', -3)","('↓', -3)","('↘', -1)","('↓', -1)","('↘', -1)","('↘', 1)","('→', 0)"
8,T,"('↓', -8)","('↓', -6)","('↓', -4)","('↘', -4)","('↓', -2)","('↘', -2)","('↘', -2)","('↓', 0)","('↘', 2)"


In [10]:
print("".join(aligned_y))
print("".join(aligned_x))

CGAGAC--G-T--
--AGACTAGTTAC


In [12]:
mat = pd.read_csv('mat_ones.csv', sep = " ", index_col=0)
x = "TGATTAGAATAC"
y = "CGAGTACTGG"
aligned_x, aligned_y, grid = needleman_wunsch(x, y, mat, 1)
print("".join(aligned_y))
print("".join(aligned_x))

CGAGT--ACTGG
TGATTAGAATAC


In [13]:
grid

,X,0,1,2,3,4,5,6,7,8,9,10
Y,,,C,G,A,G,T,A,C,T,G,G
0,,"('→', 0)","('→', -1)","('→', -2)","('→', -3)","('→', -4)","('→', -5)","('→', -6)","('→', -7)","('→', -8)","('→', -9)","('→', -10)"
1,T,"('↓', -1)","('↘', -1)","('→', -2)","('→', -3)","('→', -4)","('↘', -3)","('→', -4)","('→', -5)","('→', -6)","('→', -7)","('→', -8)"
2,G,"('↓', -2)","('↘', -2)","('↘', 0)","('→', -1)","('→', -2)","('→', -3)","('→', -4)","('→', -5)","('→', -6)","('↘', -5)","('→', -6)"
3,A,"('↓', -3)","('↘', -3)","('↓', -1)","('↘', 1)","('→', 0)","('→', -1)","('→', -2)","('→', -3)","('→', -4)","('→', -5)","('→', -6)"
4,T,"('↓', -4)","('↘', -4)","('↓', -2)","('↓', 0)","('↘', 0)","('↘', 1)","('→', 0)","('→', -1)","('→', -2)","('→', -3)","('→', -4)"
5,T,"('↓', -5)","('↘', -5)","('↓', -3)","('↓', -1)","('↘', -1)","('↘', 1)","('→', 0)","('→', -1)","('↘', 0)","('→', -1)","('→', -2)"
6,A,"('↓', -6)","('↘', -6)","('↓', -4)","('↘', -2)","('↘', -2)","('↓', 0)","('↘', 2)","('→', 1)","('→', 0)","('→', -1)","('→', -2)"
7,G,"('↓', -7)","('↘', -7)","('↘', -5)","('↓', -3)","('↘', -1)","('↓', -1)","('↓', 1)","('↘', 1)","('→', 0)","('↘', 1)","('→', 0)"
8,A,"('↓', -8)","('↘', -8)","('↓', -6)","('↘', -4)","('↓', -2)","('↘', -2)","('↘', 0)","('↘', 0)","('↘', 0)","('↓', 0)","('↘', 0)"
